In [1]:
import random

# Setup: 3 Parties with private salaries
# Let's assume salaries that sum to 220 as per expected output
salaries = {
    "Ahmed": 60,
    "Ali": 70,
    "Rashida": 90
}

parties = list(salaries.keys())
shares = {p: [] for p in parties}

# Step 1 & 2: Split and Distribute Shares
# Each person splits their salary into 3 random parts (one for each party)
for person, salary in salaries.items():
    part1 = random.randint(1, 50)
    part2 = random.randint(1, 50)
    part3 = salary - (part1 + part2) # The last part ensures the sum is correct

    # Distribute shares to Ahmed, Ali, and Rashida respectively
    shares["Ahmed"].append(part1)
    shares["Ali"].append(part2)
    shares["Rashida"].append(part3)

    print(f"{person}'s secret ({salary}) split into shares: {part1}, {part2}, {part3}")

print("-" * 30)

# Step 3: Local Sums
# Each party sums the shares they received (they don't know who sent what original amount)
local_sums = {}
for p in parties:
    local_sums[p] = sum(shares[p])
    print(f"{p} holds shares {shares[p]} and computes local sum: {local_sums[p]}")

print("-" * 30)

# Step 4: Global Sum
# Combine local sums to get the total
total_salary = sum(local_sums.values())
print(f"Final Reconstructed Total Salary: {total_salary}")

Ahmed's secret (60) split into shares: 26, 20, 14
Ali's secret (70) split into shares: 47, 2, 21
Rashida's secret (90) split into shares: 47, 13, 30
------------------------------
Ahmed holds shares [26, 47, 47] and computes local sum: 120
Ali holds shares [20, 2, 13] and computes local sum: 35
Rashida holds shares [14, 21, 30] and computes local sum: 65
------------------------------
Final Reconstructed Total Salary: 220


In [2]:
import random
from functools import reduce

# Configuration
PRIME = 257  # A small prime number for modular arithmetic
SECRET = 123 # The secret to protect
N = 5        # Total number of shares
K = 3        # Threshold to reconstruct

def generate_shares(secret, n, k, prime):
    # Coefficients: [secret, a1, a2, ..., a(k-1)]
    coeffs = [secret] + [random.randint(1, prime - 1) for _ in range(k - 1)]

    shares = []
    for x in range(1, n + 1):
        # f(x) = secret + a1*x + a2*x^2 + ... mod prime
        y = sum([coeffs[i] * (x ** i) for i in range(k)]) % prime
        shares.append((x, y))
    return shares

def reconstruct_secret(shares, prime):
    # Using Lagrange Interpolation
    x_s, y_s = zip(*shares)
    secret = 0

    for j in range(len(shares)):
        xj, yj = x_s[j], y_s[j]

        # Calculate Lagrange basis polynomial L_j(0)
        numerator = 1
        denominator = 1
        for m in range(len(shares)):
            if j == m: continue
            xm = x_s[m]
            numerator = (numerator * -xm) % prime
            denominator = (denominator * (xj - xm)) % prime

        # Modular inverse of denominator
        lagrange_term = (numerator * pow(denominator, -1, prime)) % prime
        secret = (secret + yj * lagrange_term) % prime

    return secret

# 1. Generate Shares
my_shares = generate_shares(SECRET, N, K, PRIME)
print(f"Generated {N} shares: {my_shares}")

# 2. Verify Reconstruction with k shares
subset = random.sample(my_shares, K)
print(f"Attempting reconstruction with {K} shares: {subset}")
recovered_secret = reconstruct_secret(subset, PRIME)
print(f"Recovered Secret: {recovered_secret}")

# 3. Verify failure with k-1 shares (Instructional purposes)
subset_fail = my_shares[:K-1]
print(f"Attempting reconstruction with {K-1} shares (Insufficient): {subset_fail}")
try:
    # Math allows calculation, but result is garbage/incorrect
    wrong_secret = reconstruct_secret(subset_fail, PRIME)
    print(f"Result (Incorrect): {wrong_secret}")
except:
    print("Reconstruction failed.")

Generated 5 shares: [(1, 121), (2, 23), (3, 86), (4, 53), (5, 181)]
Attempting reconstruction with 3 shares: [(3, 86), (4, 53), (1, 121)]
Recovered Secret: 123
Attempting reconstruction with 2 shares (Insufficient): [(1, 121), (2, 23)]
Result (Incorrect): 219


In [3]:
import hashlib

# Helper to "encrypt" (using hash for simplicity in this demo)
def encrypt(k1, k2, val):
    key = str(k1) + str(k2)
    return hashlib.sha256(key.encode()).hexdigest() + ":" + str(val)

# Helper to "decrypt" (check hash match)
def decrypt(k1, k2, garbled_row):
    key = str(k1) + str(k2)
    hashed = hashlib.sha256(key.encode()).hexdigest()
    if garbled_row.startswith(hashed):
        return garbled_row.split(":")[1]
    return None

# Setup: Generate random keys (Labels) for wire A and wire B
# Format: {0: KeyFor0, 1: KeyFor1}
Label_A = {0: random.getrandbits(16), 1: random.getrandbits(16)}
Label_B = {0: random.getrandbits(16), 1: random.getrandbits(16)}
Label_Out = {0: "Encrypted_0", 1: "Encrypted_1"}

print(f"Wire A Labels: {Label_A}")
print(f"Wire B Labels: {Label_B}")

# 1. Create Garbled Table for AND Gate
# Truth Table: 0&0=0, 0&1=0, 1&0=0, 1&1=1
garbled_table = []
for a in [0, 1]:
    for b in [0, 1]:
        output_val = a & b
        # Encrypt the output label using input labels
        entry = encrypt(Label_A[a], Label_B[b], Label_Out[output_val])
        garbled_table.append(entry)

# Shuffle table so position doesn't reveal logic
random.shuffle(garbled_table)

print("\n--- Computation Phase ---")
# 2. Simulation: Omar has bit 1, Nancy has bit 1 (expecting 1 AND 1 = 1)
omar_input_key = Label_A[1]
nancy_input_key = Label_B[1]

print(f"Evaluator receives keys: {omar_input_key} (from Omar), {nancy_input_key} (from Nancy)")

# 3. Evaluation: Try to decrypt rows in the table
result = None
for row in garbled_table:
    res = decrypt(omar_input_key, nancy_input_key, row)
    if res:
        result = res
        break

print(f"Computed Output: {result}")

Wire A Labels: {0: 30179, 1: 32236}
Wire B Labels: {0: 52409, 1: 27713}

--- Computation Phase ---
Evaluator receives keys: 32236 (from Omar), 27713 (from Nancy)
Computed Output: Encrypted_1
